In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import cv2
import streamlit as st
from torch.utils.data import DataLoader
from torchvision import models

In [36]:
from torchvision import transforms, datasets

# Define transformation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Ensure grayscale
    transforms.Resize((48, 48)),  # Resize to match FER-2013 resolution
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
])

# Load train dataset
train_dataset = datasets.ImageFolder(root=r"D:\train", transform=transform)

# Load test dataset
test_dataset = datasets.ImageFolder(root=r"D:\test", transform=transform)


In [38]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [39]:
# Define CNN Model
class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 6 * 6, 512)
        self.fc2 = nn.Linear(512, 7)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 6 * 6)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [40]:
# Train Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmotionCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}')

train_model()


Epoch 1/10, Loss: 1.6369820373891988
Epoch 2/10, Loss: 1.37290325117005
Epoch 3/10, Loss: 1.2288764034188404
Epoch 4/10, Loss: 1.126900766766682
Epoch 5/10, Loss: 1.0336392740895328
Epoch 6/10, Loss: 0.9442997804463308
Epoch 7/10, Loss: 0.8536146527409288
Epoch 8/10, Loss: 0.7557780730697785
Epoch 9/10, Loss: 0.6708798765605171
Epoch 10/10, Loss: 0.5888565839133443


In [41]:
# Save Model
torch.save(model.state_dict(), "emotion_cnn.pth")

